In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping



In [48]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()
##Preprocess the data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [49]:
#Encode categorical variables
labelEncoderGender=LabelEncoder()
data['Gender']=labelEncoderGender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [50]:
##Onehot encode 'Geography' column
onehotEncoderGeo=OneHotEncoder(handle_unknown='ignore')
geoEncoded=onehotEncoderGeo.fit_transform(data[['Geography']]).toarray()
geoEncoded

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [51]:
geoEncodeddf=pd.DataFrame(geoEncoded,columns=onehotEncoderGeo.get_feature_names_out(['Geography']))
geoEncodeddf

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [52]:
#Combine onehot encoded values with original data
data=pd.concat([data.drop('Geography',axis=1),geoEncodeddf],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [53]:
X= data.drop('Exited',axis=1)
y=data['Exited']

In [54]:
#Split the data in training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [55]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [56]:
#Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [57]:
#Save the encoders and scaler for later use
import pickle
with open('labelEncoderGenderHyperParameter.pkl','wb') as file:
    pickle.dump(labelEncoderGender,file)
with open('onehotEncoderGeoHyperParameter.pkl','wb') as file:
    pickle.dump(onehotEncoderGeo,file)
with open('scalerHyperParameter.pkl','wb')as file:
    pickle.dump(scaler,file)

In [58]:
#Define a function to create the model and try different paramters in KeradClassifier
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model

In [59]:
#Create a Keras Classifier
model=KerasClassifier(build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [60]:
# Define the grid search parameters
param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2],
    'epochs':[50,100]

}

In [62]:
#Perform grid search
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=1,cv=3)
grid_result=grid.fit(X_train,y_train)

ValueError: Invalid parameter layers for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(layers=1)`
Check the list of available parameters with `estimator.get_params().keys()`

In [ ]:
#Print the 